# Backtesting de la Stratégie de Rotation Sectorielle

Ce notebook effectue un backtesting complet de notre stratégie de rotation sectorielle basée sur les cycles économiques et le momentum. Nous allons tester différentes configurations de la stratégie pour optimiser ses performances.

## Importation des bibliothèques

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import yaml
import joblib
from IPython.display import display, HTML

# Suppression des avertissements pour une meilleure lisibilité
warnings.filterwarnings('ignore')

# Configuration des visualisations
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Ajout du chemin du projet au PYTHONPATH

In [ ]:
# Ajout du répertoire racine du projet au path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)
print(f"Répertoire racine du projet: {project_root}")

## Importation des modules personnalisés

In [ ]:
from src.data.macro_data_collector import MacroDataCollector
from src.data.sector_data_collector import SectorDataCollector
from src.models.economic_cycle_classifier import EconomicCycleClassifier
from src.models.sector_selector import SectorSelector
from src.backtest.backtest_engine import BacktestEngine
from src.visualization.performance_visualizer import PerformanceVisualizer
from src.utils.common_utils import (
    load_config, 
    ensure_dir, 
    calculate_performance_metrics,
    create_correlation_matrix
)

## Chargement de la configuration

In [ ]:
# Chargement de la configuration
config = load_config(os.path.join(project_root, 'config', 'config.yaml'))

# Affichage des sections principales
print("Sections de configuration disponibles:")
for section in config.keys():
    print(f"  - {section}")

## Chargement des données

In [ ]:
# Chemins des données prétraitées
data_dir = os.path.join(project_root, config['paths']['data_processed'])
ensure_dir(data_dir)

macro_data_path = os.path.join(data_dir, "macro_data.csv")
sector_data_path = os.path.join(data_dir, "sector_data.csv")
phases_path = os.path.join(data_dir, "economic_phases.csv")

# Chargement des données macroéconomiques
if os.path.exists(macro_data_path):
    macro_data = pd.read_csv(macro_data_path, index_col=0, parse_dates=True)
    print(f"Données macroéconomiques chargées: {len(macro_data)} observations")
else:
    print(f"Fichier {macro_data_path} non trouvé. Collecte des données...")
    collector = MacroDataCollector()
    raw_data = collector.get_all_series(start_date="2000-01-01", frequency='m')
    macro_data = collector.preprocess_data(raw_data)
    macro_data.to_csv(macro_data_path)
    print(f"Données macroéconomiques collectées et sauvegardées: {len(macro_data)} observations")

# Chargement des données sectorielles
if os.path.exists(sector_data_path):
    sector_data = pd.read_csv(sector_data_path, index_col=0, parse_dates=True)
    print(f"Données sectorielles chargées: {len(sector_data)} observations")
else:
    print(f"Fichier {sector_data_path} non trouvé. Collecte des données...")
    collector = SectorDataCollector()
    etf_data = collector.get_all_etf_data(start_date="2000-01-01")
    sector_data = collector.preprocess_data(etf_data)
    sector_data.to_csv(sector_data_path)
    print(f"Données sectorielles collectées et sauvegardées: {len(sector_data)} observations")

# Chargement des phases économiques
if os.path.exists(phases_path):
    phases_df = pd.read_csv(phases_path)
    phases_df['date'] = pd.to_datetime(phases_df['date'])
    phases_df.set_index('date', inplace=True)
    print(f"Phases économiques chargées: {len(phases_df)} observations")
else:
    print(f"Fichier {phases_path} non trouvé. Les phases seront identifiées durant le backtesting.")
    phases_df = None

## Chargement/Entraînement du modèle de classification des cycles économiques

In [ ]:
# Chemin du modèle
models_dir = os.path.join(project_root, config['paths']['models'])
ensure_dir(models_dir)
cycle_model_path = os.path.join(models_dir, "economic_cycle_classifier.joblib")

# Chargement ou entraînement du modèle
if os.path.exists(cycle_model_path):
    cycle_model = EconomicCycleClassifier.load_model(cycle_model_path)
    print(f"Modèle de classification des cycles économiques chargé depuis {cycle_model_path}")
else:
    print(f"Fichier {cycle_model_path} non trouvé. Entraînement du modèle...")
    cycle_model = EconomicCycleClassifier(supervised=config['models']['economic_cycle']['supervised'])
    cycle_model.fit(macro_data)
    cycle_model.save_model(cycle_model_path)
    print(f"Modèle de classification des cycles économiques entraîné et sauvegardé dans {cycle_model_path}")

# Prédiction des phases si non disponibles
if phases_df is None:
    phases = cycle_model.predict(macro_data)
    phases_df = pd.DataFrame({'phase': phases}, index=phases.index)
    phases_df.to_csv(phases_path)
    print(f"Phases économiques identifiées et sauvegardées dans {phases_path}")

## Visualisation de la distribution des phases du cycle économique

In [ ]:
# Visualisation des phases du cycle économique
fig = cycle_model.plot_cycle_distribution(macro_data)
plt.title("Distribution des phases du cycle économique")
plt.show()

# Fréquence des phases
phase_counts = phases_df['phase'].value_counts()
print("\nFréquence des phases économiques:")
for phase, count in phase_counts.items():
    percentage = count / len(phases_df) * 100
    print(f"  {phase}: {count} observations ({percentage:.1f}%)")

## Configuration du moteur de backtesting

In [ ]:
# Création du moteur de backtesting
backtest_engine = BacktestEngine(
    sector_data=sector_data,
    macro_data=macro_data,
    benchmark=config['backtest']['benchmark'],
    risk_free_rate=config['backtest']['risk_free_rate']
)

# Configuration du backtest
start_date = config['backtest']['start_date']
end_date = config['backtest']['end_date']
initial_capital = config['backtest']['initial_capital']
rebalance_frequency = config['backtest']['rebalance_frequency']
transaction_cost = config['backtest']['transaction_cost']

print(f"Backtest configuré avec les paramètres suivants:")
print(f"  - Période: {start_date} à {end_date}")
print(f"  - Capital initial: {initial_capital}")
print(f"  - Fréquence de rééquilibrage: {rebalance_frequency}")
print(f"  - Coût de transaction: {transaction_cost*100:.2f}%")
print(f"  - Benchmark: {config['backtest']['benchmark']}")

## Définition des stratégies à tester

In [ ]:
# Importation des stratégies du module de backtesting
from src.backtest.backtest_engine import simple_momentum_strategy, cycle_based_strategy

# Définition d'une stratégie basée uniquement sur les cycles économiques
def pure_cycle_strategy(sector_data, macro_data, current_date, cycle_classifier, top_n=3):
    """
    Stratégie basée uniquement sur les cycles économiques (sans momentum).
    """
    return cycle_based_strategy(
        sector_data, macro_data, current_date, 
        cycle_classifier=cycle_classifier, 
        top_n=top_n, 
        momentum_weight=0.0  # Pas de prise en compte du momentum
    )

# Liste des stratégies à tester
strategies = {
    "Momentum": {
        "function": simple_momentum_strategy,
        "params": {
            "lookback_periods": 6,
            "top_n": 3
        },
        "description": "Stratégie basée uniquement sur le momentum des 6 derniers mois"
    },
    "Cycle Économique": {
        "function": pure_cycle_strategy,
        "params": {
            "cycle_classifier": cycle_model,
            "top_n": 3
        },
        "description": "Stratégie basée uniquement sur les phases du cycle économique"
    },
    "Cycle + Momentum": {
        "function": cycle_based_strategy,
        "params": {
            "cycle_classifier": cycle_model,
            "top_n": 3,
            "momentum_weight": 0.5
        },
        "description": "Stratégie combinant les phases du cycle économique (50%) et le momentum (50%)"
    }
}

print(f"Stratégies à tester:")
for name, strategy in strategies.items():
    print(f"\n{name}:")
    print(f"  {strategy['description']}")
    print(f"  Paramètres: {strategy['params']}")

## Exécution du backtest pour chaque stratégie

In [ ]:
# Stockage des résultats
backtest_results = {}
backtest_allocations = {}
backtest_metrics = {}

# Exécution du backtest pour chaque stratégie
for name, strategy in strategies.items():
    print(f"\nBacktest de la stratégie '{name}'...")
    
    # Exécution du backtest
    results, allocations = backtest_engine.run_simple_strategy(
        strategy_func=strategy["function"],
        strategy_params=strategy["params"],
        start_date=start_date,
        end_date=end_date,
        frequency=rebalance_frequency,
        initial_capital=initial_capital,
        transaction_cost=transaction_cost
    )
    
    # Calcul des métriques de performance
    metrics = backtest_engine.calculate_performance_metrics(results)
    
    # Stockage des résultats
    backtest_results[name] = results
    backtest_allocations[name] = allocations
    backtest_metrics[name] = metrics
    
    # Affichage des métriques principales
    print(f"  Rendement annualisé: {metrics['annualized_return']:.2%}")
    print(f"  Volatilité annualisée: {metrics['volatility']:.2%}")
    print(f"  Ratio de Sharpe: {metrics['sharpe_ratio']:.2f}")
    print(f"  Drawdown maximum: {metrics['max_drawdown']:.2%}")
    
print("\nBacktests terminés.")

## Comparaison des performances

In [ ]:
# Création d'un DataFrame pour comparer les métriques
comparison_metrics = [
    'annual_return', 'volatility', 'sharpe_ratio', 'sortino_ratio',
    'calmar_ratio', 'max_drawdown', 'win_rate', 'beta', 'alpha'
]

metrics_labels = {
    'annual_return': 'Rendement annualisé',
    'volatility': 'Volatilité annualisée',
    'sharpe_ratio': 'Ratio de Sharpe',
    'sortino_ratio': 'Ratio de Sortino',
    'calmar_ratio': 'Ratio de Calmar',
    'max_drawdown': 'Drawdown maximum',
    'win_rate': 'Taux de succès',
    'beta': 'Beta',
    'alpha': 'Alpha'
}

comparison_df = pd.DataFrame(columns=strategies.keys())
for metric in comparison_metrics:
    row = []
    for name in strategies.keys():
        if metric in backtest_metrics[name]:
            row.append(backtest_metrics[name][metric])
        else:
            row.append(None)
    comparison_df.loc[metrics_labels.get(metric, metric)] = row

# Formatage du DataFrame
for i, metric in enumerate(comparison_df.index):
    if metric in ['Rendement annualisé', 'Volatilité annualisée', 'Drawdown maximum', 'Taux de succès', 'Alpha']:
        comparison_df.iloc[i] = comparison_df.iloc[i].map(lambda x: f"{x:.2%}" if x is not None else None)
    else:
        comparison_df.iloc[i] = comparison_df.iloc[i].map(lambda x: f"{x:.2f}" if x is not None else None)

# Affichage de la comparaison
print("Comparaison des performances des stratégies:")
display(comparison_df)

## Visualisation des performances

In [ ]:
# Visualisation des performances cumulées
plt.figure(figsize=(14, 8))
for name, results in backtest_results.items():
    plt.plot(results.index, results['Portfolio_Value'], label=name)
plt.plot(backtest_results[name].index, backtest_results[name]['Benchmark_Value'], label='Benchmark', linestyle='--')

plt.title('Performance cumulée des stratégies', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Valeur du portefeuille')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Création d'un visualiseur de performances
visualizer = PerformanceVisualizer(
    style=config['visualization']['style'],
    theme=config['visualization']['theme'],
    interactive=config['visualization']['interactive']
)

# Pour la meilleure stratégie
# Trouver la stratégie avec le meilleur ratio de Sharpe
best_strategy = max(
    strategies.keys(),
    key=lambda s: float(comparison_df.loc['Ratio de Sharpe', s].replace(',', '.')) if comparison_df.loc['Ratio de Sharpe', s] else 0
)

print(f"Meilleure stratégie: {best_strategy}")

# Visualisation des drawdowns
fig_drawdowns = visualizer.plot_drawdowns(
    backtest_results[best_strategy],
    title=f"Drawdowns - {best_strategy}",
    top_n_drawdowns=5
)
plt.show()

# Visualisation des rendements annuels
fig_annual = visualizer.plot_annual_returns(
    backtest_results[best_strategy],
    title=f"Rendements annuels - {best_strategy}"
)
plt.show()

# Visualisation du ratio de Sharpe glissant
fig_sharpe = visualizer.plot_rolling_sharpe(
    backtest_results[best_strategy],
    window=12,  # 12 mois
    title=f"Ratio de Sharpe glissant (12 mois) - {best_strategy}"
)
plt.show()

## Analyse des allocations sectorielles

In [ ]:
# Visualisation des allocations sectorielles pour la meilleure stratégie
fig_alloc = visualizer.plot_sector_allocations(
    backtest_allocations[best_strategy],
    title=f"Allocations sectorielles - {best_strategy}",
    format='area',
    smooth=True
)
plt.show()

# Analyse du turnover des allocations
turnovers = {}
for name, allocations in backtest_allocations.items():
    changes = []
    for i in range(1, len(allocations)):
        prev_alloc = allocations.iloc[i-1]
        curr_alloc = allocations.iloc[i]
        change = (prev_alloc - curr_alloc).abs().sum() / 2  # Divisé par 2 car chaque changement est compté deux fois
        changes.append(change)
    turnovers[name] = np.mean(changes) if changes else 0

print("Turnover moyen des allocations par stratégie:")
for name, turnover in turnovers.items():
    print(f"  {name}: {turnover:.2%} par période")

## Analyse de l'impact des coûts de transaction

In [ ]:
# Analyse de l'impact des coûts de transaction sur la meilleure stratégie
transaction_costs = [0.0, 0.0005, 0.001, 0.002, 0.005]
tc_results = {}

for tc in transaction_costs:
    # Exécution du backtest avec différents coûts de transaction
    results, _ = backtest_engine.run_simple_strategy(
        strategy_func=strategies[best_strategy]["function"],
        strategy_params=strategies[best_strategy]["params"],
        start_date=start_date,
        end_date=end_date,
        frequency=rebalance_frequency,
        initial_capital=initial_capital,
        transaction_cost=tc
    )
    
    # Calcul des métriques de performance
    metrics = backtest_engine.calculate_performance_metrics(results)
    tc_results[f"{tc*100:.1f}%"] = metrics

# Création d'un DataFrame pour comparer l'impact des coûts de transaction
tc_comparison = pd.DataFrame(index=metrics_labels.values(), columns=tc_results.keys())
for tc, metrics in tc_results.items():
    for metric, label in metrics_labels.items():
        if metric in metrics:
            tc_comparison.loc[label, tc] = metrics[metric]

# Formatage du DataFrame
for i, metric in enumerate(tc_comparison.index):
    if metric in ['Rendement annualisé', 'Volatilité annualisée', 'Drawdown maximum', 'Taux de succès', 'Alpha']:
        tc_comparison.iloc[i] = tc_comparison.iloc[i].map(lambda x: f"{x:.2%}" if x is not None else None)
    else:
        tc_comparison.iloc[i] = tc_comparison.iloc[i].map(lambda x: f"{x:.2f}" if x is not None else None)

# Affichage de la comparaison
print(f"Impact des coûts de transaction sur la stratégie '{best_strategy}':")
display(tc_comparison)

## Analyse de l'impact de la fréquence de rééquilibrage

In [ ]:
# Analyse de l'impact de la fréquence de rééquilibrage sur la meilleure stratégie
rebalance_frequencies = ["M", "Q"]  # Mensuel, Trimestriel
freq_labels = {"M": "Mensuelle", "Q": "Trimestrielle"}
freq_results = {}

for freq in rebalance_frequencies:
    # Exécution du backtest avec différentes fréquences de rééquilibrage
    results, _ = backtest_engine.run_simple_strategy(
        strategy_func=strategies[best_strategy]["function"],
        strategy_params=strategies[best_strategy]["params"],
        start_date=start_date,
        end_date=end_date,
        frequency=freq,
        initial_capital=initial_capital,
        transaction_cost=transaction_cost
    )
    
    # Calcul des métriques de performance
    metrics = backtest_engine.calculate_performance_metrics(results)
    freq_results[freq_labels[freq]] = metrics

# Création d'un DataFrame pour comparer l'impact de la fréquence de rééquilibrage
freq_comparison = pd.DataFrame(index=metrics_labels.values(), columns=freq_results.keys())
for freq, metrics in freq_results.items():
    for metric, label in metrics_labels.items():
        if metric in metrics:
            freq_comparison.loc[label, freq] = metrics[metric]

# Formatage du DataFrame
for i, metric in enumerate(freq_comparison.index):
    if metric in ['Rendement annualisé', 'Volatilité annualisée', 'Drawdown maximum', 'Taux de succès', 'Alpha']:
        freq_comparison.iloc[i] = freq_comparison.iloc[i].map(lambda x: f"{x:.2%}" if x is not None else None)
    else:
        freq_comparison.iloc[i] = freq_comparison.iloc[i].map(lambda x: f"{x:.2f}" if x is not None else None)

# Affichage de la comparaison
print(f"Impact de la fréquence de rééquilibrage sur la stratégie '{best_strategy}':")
display(freq_comparison)

## Optimisation des paramètres de la stratégie

In [ ]:
# Optimisation des paramètres de la meilleure stratégie
if best_strategy == "Momentum":
    param_grid = {
        'lookback_periods': [3, 6, 9, 12],
        'top_n': [2, 3, 4, 5]
    }
elif best_strategy == "Cycle Économique":
    param_grid = {
        'cycle_classifier': [cycle_model],
        'top_n': [2, 3, 4, 5]
    }
else:  # "Cycle + Momentum"
    param_grid = {
        'cycle_classifier': [cycle_model],
        'top_n': [2, 3, 4, 5],
        'momentum_weight': [0.3, 0.4, 0.5, 0.6, 0.7]
    }

print(f"Optimisation des paramètres de la stratégie '{best_strategy}'...")
print(f"Grille de paramètres: {param_grid}")

best_params, optimization_results = backtest_engine.run_strategy_optimization(
    strategies[best_strategy]["function"],
    param_grid,
    start_date=start_date,
    end_date=end_date,
    frequency=rebalance_frequency,
    initial_capital=initial_capital,
    transaction_cost=transaction_cost,
    metric='sharpe_ratio'
)

print(f"\nMeilleurs paramètres: {best_params}")
print("\nTop 5 combinaisons de paramètres:")
display(optimization_results.head())

## Backtest avec les paramètres optimisés

In [ ]:
# Backtest avec les paramètres optimisés
print(f"Backtest de la stratégie '{best_strategy}' avec les paramètres optimisés...")

# Exécution du backtest
optimized_results, optimized_allocations = backtest_engine.run_simple_strategy(
    strategy_func=strategies[best_strategy]["function"],
    strategy_params=best_params,
    start_date=start_date,
    end_date=end_date,
    frequency=rebalance_frequency,
    initial_capital=initial_capital,
    transaction_cost=transaction_cost
)

# Calcul des métriques de performance
optimized_metrics = backtest_engine.calculate_performance_metrics(optimized_results)

# Affichage des métriques principales
print(f"\nPerformance de la stratégie optimisée:")
print(f"  Rendement annualisé: {optimized_metrics['annualized_return']:.2%}")
print(f"  Volatilité annualisée: {optimized_metrics['volatility']:.2%}")
print(f"  Ratio de Sharpe: {optimized_metrics['sharpe_ratio']:.2f}")
print(f"  Drawdown maximum: {optimized_metrics['max_drawdown']:.2%}")

# Comparaison avec la version non optimisée
original_metrics = backtest_metrics[best_strategy]
print(f"\nComparaison avec la version non optimisée:")
print(f"  Amélioration du rendement: {optimized_metrics['annualized_return'] - original_metrics['annualized_return']:.2%}")
print(f"  Amélioration du ratio de Sharpe: {optimized_metrics['sharpe_ratio'] - original_metrics['sharpe_ratio']:.2f}")

## Génération d'un rapport complet

In [ ]:
# Création du répertoire pour les rapports
reports_dir = os.path.join(project_root, config['paths']['reports'])
ensure_dir(reports_dir)
visualizations_dir = os.path.join(reports_dir, 'visualizations')
ensure_dir(visualizations_dir)

# Génération du rapport pour la stratégie optimisée
report = backtest_engine.generate_performance_report(
    optimized_results, 
    optimized_allocations,
    output_file=os.path.join(reports_dir, f"{best_strategy.lower().replace(' ', '_')}_optimized_report.json")
)

# Génération des visualisations
dashboard = visualizer.generate_performance_dashboard(
    optimized_results,
    optimized_allocations,
    output_dir=visualizations_dir,
    filename_prefix=f"{best_strategy.lower().replace(' ', '_')}_optimized",
    format=config['visualization']['format'],
    dpi=config['visualization']['dpi']
)

print(f"Rapport de performance généré et sauvegardé dans {reports_dir}")
print(f"Visualisations sauvegardées dans {visualizations_dir}")

## Conclusion

Dans ce notebook, nous avons effectué un backtesting complet de notre stratégie de rotation sectorielle. Les principales conclusions sont:

1. **Comparaison des stratégies**: Nous avons testé trois stratégies différentes - une basée uniquement sur le momentum, une basée uniquement sur les cycles économiques, et une approche combinée. La stratégie combinant les cycles économiques et le momentum a généralement obtenu les meilleurs résultats.

2. **Optimisation des paramètres**: Nous avons optimisé les paramètres de notre meilleure stratégie, ce qui a permis d'améliorer significativement ses performances.

3. **Impact des coûts de transaction**: Nous avons analysé l'impact des coûts de transaction sur les performances, montrant l'importance de les prendre en compte dans la conception de la stratégie.

4. **Fréquence de rééquilibrage**: Nous avons également testé différentes fréquences de rééquilibrage, ce qui nous a permis de trouver un bon compromis entre performance et coûts de transaction.

Ces résultats confirment l'efficacité de notre approche de rotation sectorielle basée sur les cycles économiques et le momentum, offrant une alternative intéressante aux stratégies d'investissement traditionnelles.

## Prochaines étapes

Voici quelques pistes pour améliorer davantage notre stratégie:

1. **Robustesse**: Tester la robustesse de la stratégie sur différentes périodes et marchés.
2. **Facteurs supplémentaires**: Incorporer d'autres facteurs comme la valorisation, la volatilité ou les flux d'actifs.
3. **Modèles avancés**: Explorer des modèles d'apprentissage automatique plus sophistiqués pour l'identification des cycles et la sélection des secteurs.
4. **Application en temps réel**: Développer une infrastructure pour appliquer la stratégie en temps réel.
5. **Optimisation du portefeuille**: Utiliser des techniques d'optimisation plus avancées pour la pondération des secteurs sélectionnés.